In [229]:
import pandas as pd
import plotly.graph_objects as go

In [230]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [231]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,21,22,22,22,24,24,40,40,74,84
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,51,55,59,64,70,76,89,104,123,146
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,54,60,74,87,90,139,201,230,264,302
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,2,39,39,53,75,88,113,133,164,188
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,1,2,2,3,3,3


In [232]:
deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,2
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1,1,2,2,2,2,2,4,5,5
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,4,4,7,9,11,15,17,17,19,21
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
def pivot_date(df):
    df_new = df.melt(id_vars=['Province/State','Country/Region','Lat','Long'],value_name='Cases',var_name='Date')
    df_new = df_new.set_index(['Country/Region', 'Province/State', 'Date'])
    return df_new

confirmed_new = pivot_date(confirmed)
deaths_new = pivot_date(deaths)

In [234]:
confirmed_new.head()

,,,Lat,Long,Cases
Country/Region,Province/State,Date,,,
Afghanistan,NaN,1/22/20,33.0000,65.0000,0
Albania,NaN,1/22/20,41.1533,20.1683,0
Algeria,NaN,1/22/20,28.0339,1.6596,0
Andorra,NaN,1/22/20,42.5063,1.5218,0
Angola,NaN,1/22/20,-11.2027,17.8739,0


In [235]:
deaths_new.head()

,,,Lat,Long,Cases
Country/Region,Province/State,Date,,,
Afghanistan,NaN,1/22/20,33.0000,65.0000,0
Albania,NaN,1/22/20,41.1533,20.1683,0
Algeria,NaN,1/22/20,28.0339,1.6596,0
Andorra,NaN,1/22/20,42.5063,1.5218,0
Angola,NaN,1/22/20,-11.2027,17.8739,0


In [236]:
def country_level(df, old_name, new_name):
    df_new = df.groupby(['Country/Region','Date'])['Cases'].sum().reset_index()
    df_new = df_new.set_index(['Country/Region','Date'])
    df_new.index=df_new.index.set_levels([df_new.index.levels[0], pd.to_datetime(df_new.index.levels[1])])
    df_new=df_new.sort_values(['Country/Region','Date'],ascending=True)
    df_new=df_new.rename(columns={old_name:new_name})
    return df_new
    
confirmed_country = country_level(confirmed_new, 'Cases', 'Confirmed Cases')
deaths_country = country_level(deaths_new, 'Cases', 'Deaths')

In [237]:
confirmed_country.head()

Confirmed Cases
Country/Region Date                       
Afghanistan    2020-01-22                0
               2020-01-23                0
               2020-01-24                0
               2020-01-25                0
               2020-01-26                0

In [238]:
deaths_country.head()

Deaths
Country/Region Date              
Afghanistan    2020-01-22       0
               2020-01-23       0
               2020-01-24       0
               2020-01-25       0
               2020-01-26       0

In [239]:
country = pd.merge(confirmed_country, deaths_country, how='left', left_index=True, right_index=True)

In [240]:
country.tail()

Confirmed Cases  Deaths
Country/Region Date                               
Zimbabwe       2020-03-21                3       0
               2020-03-22                3       0
               2020-03-23                3       1
               2020-03-24                3       1
               2020-03-25                3       1

In [241]:
canada = country.reset_index()
canada = canada[canada['Country/Region'] == 'Canada']
canada = canada[canada['Date'] >= '2020-02-21']
canada['Deaths%'] = (canada['Deaths'] / canada['Confirmed Cases']) * 100
canada['Percent Change New Confirmed Cases'] = canada['Confirmed Cases'].pct_change()
print(canada.tail())

     Country/Region       Date  Confirmed Cases  Deaths   Deaths%  \
1915         Canada 2020-03-21             1277      19  1.487862   
1916         Canada 2020-03-22             1469      21  1.429544   
1917         Canada 2020-03-23             2088      25  1.197318   
1918         Canada 2020-03-24             2790      26  0.931900   
1919         Canada 2020-03-25             3251      30  0.922793   

      Percent Change New Confirmed Cases  
1915                            0.354189  
1916                            0.150352  
1917                            0.421375  
1918                            0.336207  
1919                            0.165233  


In [242]:
fig = go.Figure(data = go.Scatter(x=canada.Date, y=canada['Confirmed Cases']), layout_title_text = 'Confirmed Cases in Canada')

In [243]:
fig.show()

In [244]:
fig = go.Figure(data = go.Scatter(x=canada.Date, y=canada['Deaths']), layout_title_text = 'Deaths in Canada')

In [245]:
fig.show()

In [246]:
fig = go.Figure(data = go.Scatter(x=canada.Date, y=canada['Deaths%']), layout_title_text = 'Percentage Deaths in Canada')

In [247]:
fig.show()

In [248]:
fig = go.Figure(data = go.Scatter(x=canada.Date, y=canada['Percent Change New Confirmed Cases']), layout_title_text = 'Percentage Change New Cases in Canada')
fig.show()